In [1]:
import string

class TOKEN:
    def __init__(self, classe, lexema, tipo):
        self.classe = classe
        # Transformando o parâmetro lexema obtido como lista para uma string
        self.lexema = ''.join(lexema)
        self.tipo = tipo
    
    # Define o que aparece quando se utiliza print(token)
    def __str__(self):
        return f'Classe={self.classe}, lexema={self.lexema}, Tipo={self.tipo}'

'''
Nova exception herdando de Exception para diferenciar quando utiliza try... except
'''
# 
class AlphabetError(Exception):
    pass

'''
Tabela inicializada com palavras reservadas e ids encontrados no programa fonte

'''
TABELA_DE_SIMBOLOS = [
    TOKEN('inicio', 'inicio', 'inicio'),
    TOKEN('varinicio', 'varinicio', 'varinicio'),
    TOKEN('varfim', 'varfim', 'varfim'),
    TOKEN('escreva', 'escreva', 'escreva'),
    TOKEN('leia', 'leia', 'leia'),
    TOKEN('se', 'se', 'se'),
    TOKEN('entao', 'entao', 'entao'),
    TOKEN('fimse', 'fimse', 'fimse'),
    TOKEN('fim', 'fim', 'fim'),
    TOKEN('inteiro', 'inteiro', 'inteiro'),
    TOKEN('literal', 'literal', 'literal'),
    TOKEN('real', 'real', 'real')
]

'''
Dicionário de dicionários representando a tabela de transição do autômato
Para mudar de estado utiliza-se estado = tabela_de_transicao[estado atual][chave] na qual as chaves são os símbolos no
estado atual para mudar de estado
'''
tabela_de_transicao = {
    0: {
        'letra': 1,
        'digito': 2,
        '"': 11,
        '{': 13,
        'eof': 15,
        '=': 16,
        '>': 17,
        '<': 18,
        '+': 20,
        '-': 20,
        '*': 20,
        '/': 20,
        '(': 21,
        ')': 22,
        ';': 23,
        ',': 24
    },
    1: {
        'letra': 1,
        'digito': 1,
        '_': 1
    },
    2: {
        'digito': 2,
        '.': 3,
        'E': 8,
        'e': 8
    },
    3: {
        'digito': 4
    },
    4: {
        'digito': 4,
        'e': 5,
        'E': 5
    },
    5: {
        '+': 6,
        '-': 6,
        'digito': 7
    },
    6: {
        'digito': 7
    },
    7: {
        'digito': 7
    },
    8: {
        '+': 9,
        '-': 6,
        'digito': 10
    },
    9: {
        'digito': 10
    },
    10: {
        'digito': 10
    },
    11: {
        'curinga': 11,
        '"': 12
    },
    12: {},
    13: {
        'curinga': 13,
        '}': 14
    },
    14: {},
    15: {},
    16: {},
    17: {
        '=': 16
    },
    18: {
        '=': 16,
        '>': 16,
        '-': 19
    },
    19: {},
    20: {},
    21: {},
    22: {},
    23: {},
    24: {}
}

'''
Quando o autômato termina uma cadeia em algum desses estados sabemos qual tipo de token estamos lidando
'''
estados_finais = {
    0: 'estado inicial',
    1: 'id', # ou palavra reservada
    2: 'Num',
    4: 'Num',
    7: 'Num',
    10: 'Num',
    12: 'Lit',
    14: 'Comentário',
    15: 'EOF',
    16: 'OPR',
    17: 'OPR',
    18: 'OPR',
    19: 'ATR',
    20: 'OPA',
    21: 'AB_P',
    22: 'FC_P',
    23: 'PT_V',
    24: 'VIR'
}

'''
Algumas listas úteis
'''
letras = list(string.ascii_letters)
digitos = list(string.digits)
delimitadores = [' ', '\n', '\t']
alfabeto = list(letras + digitos + delimitadores + [
    ',', ';', ':', '.', '!', '?', '\\', '*', '+', '-', '/', '(', ')', '{', '}',
    '[', ']', '<', '>', '=', "'", '"'
])

In [2]:
'''
Essa função recebe o estado e o lexema para classificar e retornar o novo token
'''
def retorna_TOKEN(estado, lexema):
    # Verifica a cadeia completa por string sem fim ou caractere fora do alfabeto
    for l in lexema:
        if (estado == 11 and l == '\n') or (l not in alfabeto):
            return TOKEN('ERROR', lexema, None)
        
    # Se estado == 1 então é um id ou uma palavra reservada
    if estado == 1:
        # Se já estiver na tabela de símbolos então o token da tabela de símbolos é retornado
        for token in TABELA_DE_SIMBOLOS:
            # Como lexema é uma lista é necessário usar join para comparar com uma string
            if token.lexema == ''.join(lexema):
                return token
        # Caso não tenha retornado então significa que foi encontrado um id que não está na tabela de símbolos
        novo_id = TOKEN('id', lexema, None)
        TABELA_DE_SIMBOLOS.append(novo_id)
        return novo_id
    
    # Definindo o tipo do token de acordo com a descrição do trabalho
    elif estado == 2 or estado == 10:
        tipo = 'inteiro'
        
    elif estado == 4 or estado == 7:
        tipo = 'real'
        
    elif estado == 12:
        tipo = 'literal'
        
    else:
        tipo = None
    
    # Saída padrão caso não seja um id ou palavra reservada
    return TOKEN(estados_finais[estado], lexema, tipo)

'''
Essa função cria a chave para a transição de estados na tabela de transições de acordo com o estado atual e o caractere
recebido.

Utiliza-se as chaves "letra", "digito" e "curinga" ao invés do próprio caractere para não confundir com outros caracteres
iguais e para evitar a criação de um estado para cada letra e dígito.
'''
def chave(caractere, estado):
        if (estado == 2 or estado == 4) and (caractere == 'e' or caractere == 'E'):
            chave = 'e'
        elif (estado == 11 and caractere != '"') or (estado == 13 and caractere != '}'):
            chave = 'curinga'
        elif caractere in letras:
            chave = 'letra'
        elif caractere in digitos:
            chave = 'digito'
        else:
            chave = caractere
            
        return chave


'''
Analisador léxico.

Recebe o código fonte como uma string e o lê caractere a caractere, unindo-os em lexemas que serão reconhecidos como
pertencentes a um padrão.
'''    
def SCANNER(fonte, posicao, linha, coluna):
    # Estado inicial
    estado = 0
    # Lexema é uma string, mas está sendo tratada como lista para facilitar operações com string
    lexema = []
    
    # Lê todos os caracteres do fonte
    while posicao <= len(fonte):
        try:
            # Se não pertence ao alfabeto gera uma exceção
            if fonte[posicao] not in alfabeto:
                raise AlphabetError
            # Caso pertença ao alfabeto realiza a transição de estados
            estado = tabela_de_transicao[estado][chave(fonte[posicao], estado)]
            
        # Foi encontrado um caractere que não pertence ao alfabeto
        except AlphabetError:
            # Caso exista um lexema sendo lido quando esse caractere foi encontrado retorna o token desse lexema primeiro
            if (len(lexema) > 0) and (estado in estados_finais):
                return retorna_TOKEN(estado, lexema), posicao, linha, coluna
            # Caso contrário mostra onde o caractere foi encontrado e continua análise
            print(f'ERRO LÉXICO - Caractere inválido na linguagem: {fonte[posicao]}. Linha {linha}, coluna {coluna}')
            lexema.append(fonte[posicao])
        
        # Quebra de padrão: Foi feita uma tentativa de fazer uma transição inexistente naquele estado
        except KeyError:
            # Caso haja um lexema e está em um estado final, então é porque já chegou ao fim
            if (len(lexema)) > 0 and (estado in estados_finais.keys()):
                return retorna_TOKEN(estado, lexema), posicao, linha, coluna
            # Caso não esteja em um estado final (Ex.: 1e falta 1 dígito após o e)
            elif estado not in estados_finais.keys():
                print(f'ERRO LÉXICO - Cadeia {"".join(lexema)} não acaba em um estado final. Linha {linha}, coluna {coluna}')                        
                return TOKEN('ERROR', lexema, None), posicao, linha, coluna
        # Chegou ao fim da string fonte
        except IndexError:
            # Há um lexema que deve ser retornado antes do EOF
            if len(lexema) > 0:
                return retorna_TOKEN(estado, lexema), posicao, linha, coluna
            # Sai do loop e retorna o EOF
            else: 
                break
        
        # Ocorreu a transição normalmente
        else:
            # Se o delimitador for \n então atualiza linha e coluna
            if fonte[posicao] == '\n':
                linha = linha + 1
                coluna = 0
                if estado == 11:
                    # EOL = End of line
                    print(f'ERRO LÉXICO - EOL encontrado enquanto verificando string literal. Linha {linha}, coluna {coluna}')
            # Junta caractere ao lexema
            # Não ignora delimitadores dentro de comentários e literais
            if (fonte[posicao] not in delimitadores) or (estado == 11 or estado == 13):
                lexema.append(fonte[posicao])
        # Atualiza contadores
        finally:
            posicao = posicao + 1
            coluna = coluna + 1
            
    # Fim do arquivo fonte
    return TOKEN('EOF', 'EOF', None), posicao, linha, coluna

In [3]:
'''
Função main().

Lê o arquivo e faz um loop enquanto chama o SCANNER().
'''
def main():
    with open('teste.txt') as file:
        fonte = file.read()
        posicao = 0
        linha = 1
        coluna = 1
        
        while posicao <= len(fonte):
            token, posicao, linha, coluna = SCANNER(fonte, posicao, linha, coluna)
            print(token)

In [4]:
main()

Classe=Num, lexema=1, Tipo=inteiro
Classe=Num, lexema=3, Tipo=inteiro
ERRO LÉXICO - EOL encontrado enquanto verificando string literal. Linha 2, coluna 0
Classe=Lit, lexema="teste 1 2 
3", Tipo=literal
Classe=EOF, lexema=EOF, Tipo=None
